In [1]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import paho.mqtt.client as mqtt

# This function defines what to do when we connect to the broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    # We subscribe to this particular data. There may be other data published by the same subscriber or broker.
    client.subscribe("myroom/temperature")

# This function defines what to do when we receives a message from the brokder
def on_message(client, userdata, msg):
    print("Received myroom/temperature: ", msg.payload.decode())

# We create a client as the data subscriber and specify its actions for particular events
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

# Now, we connect to the data broker.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# As a simple example, we just keep the data listening/receiving on and on...
mqttc.loop_forever()